# Network Automation

Neste notebook estão presentes vários scripts que podem ser utilizados em automação de redes. 

Nota que estes scripts só funcionam para ExtremeXOS. Caso pretenda mudar algum script tenha em consideração que os comandos possam ser diferentes de fabricante para fabricante.

## Adicionar VLANs ao STP

In [ ]:
import exsh
import re

try:
    # Get the list of VLANs
    output = exsh.clicmd("show vlan", capture=True)
    vlans = re.findall(r"^(\d+)\s+", output, re.MULTILINE)

    for vlan in vlans:
        try:
            # Check if VLAN is already in STP
            stp_output = exsh.clicmd("show stp", capture=True)
            if f"VLAN {vlan}" not in stp_output:
                try:
                    # Add VLAN to STP
                    exsh.clicmd(f"configure stp vlan {vlan}")
                    print(f"[+] Successfully added VLAN {vlan} to STP")
                except Exception as e:
                    print(f"[-] Error adding VLAN {vlan} to STP: {e}")
            else:
                print(f"[*] VLAN {vlan} is already in STP")
        except Exception as e:
            print(f"[-] Error checking STP status for VLAN {vlan}: {e}")
except Exception as e:
    print(f"[-] Error retrieving VLANs: {e}")
    exit()


## Configurações de Backup

In [ ]:
import exsh
import os
import datetime
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define USB path
usbpath = "/usr/local/ext/"

try:
    # Retrieve the configuration from the device
    configs = exsh.clicmd("show configuration", capture=True).strip()

    # Debug: Check raw output
    logging.info(f"Config Output Length: {len(configs)} characters")

    if not configs:
        logging.error("No configuration data retrieved. The output is empty.")
        exit()

    # Generate a timestamped filename
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_filename = f"exos_backup_config_{timestamp}.txt"
    backup_filepath = os.path.join(usbpath, backup_filename)

    # Check if the USB path exists
    if not os.path.exists(usbpath):
        logging.error(f"USB path '{usbpath}' does not exist. Ensure the storage is mounted.")
        exit()

    # Ask for user confirmation before saving
    confirm = input(f"Do you want to save the configuration backup to '{backup_filename}'? (y/n): ").strip().lower()

    if confirm == 'y':
        # Write configuration to the backup file
        with open(backup_filepath, 'w', encoding='utf-8') as file:
            file.write(configs)
            logging.info(f"Configuration backup saved to {backup_filepath}")
    else:
        logging.warning("Backup operation canceled.")

except PermissionError:
    logging.error(f"Permission denied: Unable to write to '{usbpath}'. Check write permissions.")
except Exception as e:
    logging.error(f"Error retrieving configurations: {e}")
    exit()


## Shut de Rogue DHCP Servers

In [ ]:
import exsh
import re

# Run DHCP snooping check
dhcp_output = exsh.clicmd("show ip-security dhcp-snooping violations", capture=True)

# Detect unauthorized DHCP servers
rogue_servers = []
for line in dhcp_output.split("\n"):
    match = re.search(r"(\S+)\s+(\S+)\s+Rogue", line)  # Extract IP and port
    if match:
        ip, port = match.groups()
        rogue_servers.append((ip, port))

# Disable rogue servers if found
if rogue_servers:
    for ip, port in rogue_servers:
        exsh.clicmd(f"disable port {port}")
        print(f"Rogue DHCP server detected on {port} (IP: {ip}) - Port Disabled!")
else:
    print("No rogue DHCP servers detected.")
